In [1]:
import pandas as pd
from Bio import SeqIO
from io import StringIO
import os
import subprocess
import ray
from subprocess import Popen, PIPE, STDOUT

In [2]:
input_fasta= "/home/infosebi/Documents/Programmieren/MastersProject/clusters.fasta"
output_folder= "/home/infosebi/Documents/Programmieren/MastersProject/clusters"
mafft_folder="/home/infosebi/Documents/Programmieren/MastersProject/genomes/aligned"
tree_folder="/home/infosebi/Documents/Programmieren/MastersProject/genomes/trees"

In [3]:
cluster_df = pd.read_csv('genomes/reclusters.txt', sep='\t', header=None)

In [4]:
prot_lists= []
prot_old = ''
for index, row in cluster_df.iterrows():
    prot = row[0]
    if prot != prot_old:
        if index > 0:
            prot_lists.append(prot_list)
        prot_list = []
        prot_list.append(row[1])
    else:
        prot_list.append(row[1])
    prot_old = prot    

In [5]:
seq_dict = {}
for record in SeqIO.parse("clusters.fasta", "fasta"):
        seq_dict[record.id]=record

In [6]:
cluster_list = []
for cluster in prot_lists:
    helper_list = []
    for protein in cluster:
        helper_list.append(seq_dict[protein])
    cluster_list.append(helper_list)


In [7]:
ray.init()

2023-11-15 13:21:11,949	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.8
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [8]:
@ray.remote
def maf_tree(cluster):
    if len(cluster) >= 2:
        out_handle = StringIO()
        SeqIO.write(cluster, out_handle, "fasta")
        fasta_data = out_handle.getvalue()
        mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        output_file = f'{mafft_folder}/{cluster[0].id}.fasta'
        with open(output_file, 'w') as output_handle:
            output_handle.write(stdout_data)
    if len(cluster) >= 3:
            subprocess.run(f"fasttree < {mafft_folder}/{cluster[0].id}.fasta >  {tree_folder}/{cluster[0].id}.tre",shell=True)

In [9]:
futures = [maf_tree.remote(cluster) for cluster in cluster_list]
ray.get(futures)

2023-11-15 13:32:16,652	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(maf_tree pid=211136) FastTree Version 2.1.11 Double precision (No SSE3)
(maf_tree pid=211136) Alignment: standard input
(maf_tree pid=211136) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
(maf_tree pid=211136) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
(maf_tree pid=211136) TopHits: 1.00*sqrtN close=default refresh=0.80
(maf_tree pid=211136) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
(maf_tree pid=211136) Initial topology in 0.00 seconds
(maf_tree pid=211136) Refining topology: 4 rounds ME-NNIs, 2 rounds ME-SPRs, 2 rounds ML-NNIs
(maf_tree pid=211136) Total branch-length 0.692 after 0.00 sec
(maf_tree pid=211136) ML-NNI round 1: LogLk = -321.000 NNIs 0 max delta 0.00 Time 0.01
(maf_tree pid=211136) Turning off heuristics for final round of ML NNIs
(maf_tree pid=211136) Switched to using 20 rate cat

(maf_tree pid=211142)       0.11 seconds: ME NNI round 7 of 9, 1 of 3 splits
(maf_tree pid=211142) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 2x across cluster]
(maf_tree pid=211142) Alignment: standard input [repeated 2x across cluster]
(maf_tree pid=211142) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 2x across cluster]
(maf_tree pid=211142) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 2x across cluster]
(maf_tree pid=211142) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 2x across cluster]
(maf_tree pid=211142) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 2x across cluster]
(maf_tree pid=211142) Initial topology in 0.06 seconds [repeated 2x across cluster]
(maf_tree pid=211142) Refining topology: 9 rounds ME-NNIs, 2 rounds ME-SPRs, 5 rounds ML-NNIs [repeated 2x across cluster]
(maf_tree pid=211142) Total branch-length 1.119 after 0.12 sec [repeated 2x acr

(maf_tree pid=211141) ML-NNI round 3: LogLk = -349.841 NNIs 0 max delta 0.00 Time 0.41 (final) [repeated 6x across cluster]
(maf_tree pid=211141)       0.10 seconds: Site likelihoods with rate category 2 of 20 [repeated 3x across cluster]
(maf_tree pid=211140)       0.23 seconds: ML NNI round 3 of 4, 1 of 2 splits [repeated 2x across cluster]
(maf_tree pid=211141) Turning off heuristics for final round of ML NNIs
(maf_tree pid=211138)       0.51 seconds: ML Lengths 1 of 2 splits
(maf_tree pid=211136) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 5x across cluster]
(maf_tree pid=211136) Alignment: standard input [repeated 5x across cluster]
(maf_tree pid=211136) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 5x across cluster]
(maf_tree pid=211136) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 5x across cluster]
(maf_tree pid=211136) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 5x across cluster]
(maf

(maf_tree pid=211140) ML-NNI round 3: LogLk = -208.969 NNIs 0 max delta 0.00 Time 0.02 (final) [repeated 8x across cluster]
(maf_tree pid=211143)       9.40 seconds: Site likelihoods with rate category 20 of 20 [repeated 27x across cluster]
(maf_tree pid=211143)       9.59 seconds: ML NNI round 2 of 8, 1 of 14 splits [repeated 4x across cluster]
(maf_tree pid=211140) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
(maf_tree pid=211142)       0.61 seconds: ML Lengths 1 of 3 splits
(maf_tree pid=211137) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 10x across cluster]
(maf_tree pid=211137) Alignment: standard input [repeated 10x across cluster]
(maf_tree pid=211137) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 10x across cluster]
(maf_tree pid=211137) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 10x across cluster]
(maf_tree pid=211137) TopHits: 1.00*sqrtN close=default refresh=0.80 [r

(maf_tree pid=211136) Total branch-length 1.004 after 0.03 sec
(maf_tree pid=211142) Turning off heuristics for final round of ML NNIs
(maf_tree pid=211139) ML-NNI round 3: LogLk = -2812.618 NNIs 0 max delta 0.00 Time 0.80 (final) [repeated 6x across cluster]
(maf_tree pid=211139)       0.49 seconds: Site likelihoods with rate category 14 of 20 [repeated 10x across cluster]
(maf_tree pid=211139)       0.10 seconds: ML NNI round 1 of 4, 1 of 2 splits [repeated 7x across cluster]
(maf_tree pid=211139)       0.79 seconds: ML Lengths 1 of 2 splits [repeated 4x across cluster]
(maf_tree pid=211141) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 6x across cluster]
(maf_tree pid=211141) Alignment: standard input [repeated 6x across cluster]
(maf_tree pid=211141) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 6x across cluster]
(maf_tree pid=211141) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 6x across cluster]
(maf_tr

...

(maf_tree pid=211137) Total time: 10.90 seconds Unique: 6/10 Bad splits: 0/3
(maf_tree pid=211139) ML-NNI round 3: LogLk = -1126.111 NNIs 0 max delta 0.00 Time 0.96 (final) [repeated 2x across cluster]
(maf_tree pid=211143) Optimize all lengths: LogLk = -1919.336 Time 3.88 [repeated 3x across cluster]
(maf_tree pid=211139)       0.57 seconds: Site likelihoods with rate category 16 of 20 [repeated 9x across cluster]
(maf_tree pid=211143)       2.62 seconds: ML NNI round 3 of 5, 1 of 4 splits [repeated 5x across cluster]
(maf_tree pid=211143)       3.35 seconds: ML Lengths 1 of 4 splits [repeated 3x across cluster]
(maf_tree pid=211143) Turning off heuristics for final round of ML NNIs (converged) [repeated 2x across cluster]
(maf_tree pid=211136) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 2x across cluster]
(maf_tree pid=211136) Alignment: standard input [repeated 2x across cluster]
(maf_tree pid=211136) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1

(maf_tree pid=211139) Total time: 4.75 seconds Unique: 6/10 Bad splits: 0/3 [repeated 6x across cluster]
(maf_tree pid=211136) ML-NNI round 3: LogLk = -1651.917 NNIs 0 max delta 0.00 Time 1.68 (final) [repeated 6x across cluster]
(maf_tree pid=211141)       0.18 seconds: ME NNI round 1 of 18, 1 of 19 splits
(maf_tree pid=211141)       1.40 seconds: ME NNI round 7 of 18, 1 of 19 splits
(maf_tree pid=211136) Optimize all lengths: LogLk = -1651.917 Time 1.92 [repeated 4x across cluster]
(maf_tree pid=211136)       0.72 seconds: Site likelihoods with rate category 20 of 20 [repeated 7x across cluster]
(maf_tree pid=211136)       1.18 seconds: ML NNI round 3 of 5, 1 of 4 splits [repeated 4x across cluster]
(maf_tree pid=211141)       2.19 seconds: ME NNI round 13 of 18, 1 of 19 splits
(maf_tree pid=211136)       1.68 seconds: ML Lengths 1 of 4 splits [repeated 2x across cluster]
(maf_tree pid=211143) Turning off heuristics for final round of ML NNIs (converged) [repeated 3x across cluster]


(maf_tree pid=211140) Total time: 1.61 seconds Unique: 4/10 Bad splits: 0/1
(maf_tree pid=211142) Total time: 1.61 seconds Unique: 4/10 Bad splits: 0/1 [repeated 8x across cluster]
(maf_tree pid=211137) ML-NNI round 3: LogLk = -1064.349 NNIs 0 max delta 0.00 Time 0.33 (final) [repeated 5x across cluster]
(maf_tree pid=211140) Turning off heuristics for final round of ML NNIs (converged)
(maf_tree pid=211143) Optimize all lengths: LogLk = -1535.662 Time 0.63 [repeated 7x across cluster]
(maf_tree pid=211143)       0.30 seconds: Site likelihoods with rate category 13 of 20 [repeated 32x across cluster]
(maf_tree pid=211140)       1.69 seconds: ML NNI round 2 of 6, 1 of 5 splits [repeated 12x across cluster]
(maf_tree pid=211143)       0.54 seconds: ML Lengths 1 of 2 splits [repeated 4x across cluster]
(maf_tree pid=211136) Turning off heuristics for final round of ML NNIs (converged) [repeated 7x across cluster]
(maf_tree pid=211137) FastTree Version 2.1.11 Double precision (No SSE3) [re

(maf_tree pid=211143) Total time: 2.60 seconds Unique: 9/20 Bad splits: 0/6 [repeated 6x across cluster]
(maf_tree pid=211138) ML-NNI round 3: LogLk = -1422.354 NNIs 0 max delta 0.00 Time 2.48 (final) [repeated 7x across cluster]
(maf_tree pid=211138)       0.24 seconds: ML NNI round 1 of 5, 1 of 3 splits
(maf_tree pid=211136) Optimize all lengths: LogLk = -972.168 Time 0.14 [repeated 8x across cluster]
(maf_tree pid=211140)       0.16 seconds: Site likelihoods with rate category 1 of 20 [repeated 12x across cluster]
(maf_tree pid=211142)       0.24 seconds: ML NNI round 1 of 5, 1 of 3 splits [repeated 8x across cluster]
(maf_tree pid=211140)       0.31 seconds: ML Lengths 1 of 2 splits [repeated 5x across cluster]
(maf_tree pid=211139) Turning off heuristics for final round of ML NNIs (converged) [repeated 7x across cluster]
(maf_tree pid=211143) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 6x across cluster]
(maf_tree pid=211143) Alignment: standard input [repeated 6x

LocalRayletDiedError: The task's local raylet died. Check raylet.out for more information.

In [ ]:
#create zip files from genome
#include mafft_alignments
#include tree folder